In [2]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import imblearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import Counter

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [4]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("Resources/lending_data.csv")
df_lending_data = pd.read_csv(file_path)

# Review the DataFrame
df_lending_data.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [5]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_lending_data["loan_status"].ravel()


# Separate the X variable, the features
X = df_lending_data.copy()
X.drop("loan_status", axis=1, inplace=True)

In [6]:
# Review the y variable Series
y[:5]

array([0, 0, 0, 0, 0])

In [7]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [8]:
# Check the balance of our target values
df_lending_data["loan_status"].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [9]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [10]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [11]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9921240885954051
Testing Data Score: 0.9918489475856377


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [12]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
19379,0,0
19380,0,0
19381,0,0
19382,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [13]:
# Print the balanced_accuracy score of the model
bal_acc=balanced_accuracy_score(y_test, predictions)
bal_acc

0.9520479254722232

In [14]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18663,   102],
       [   56,   563]])

In [15]:
# Print the classification report for the model
testing_report = classification_report(y_test, predictions)
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** In summary, the logistic regression model performed very well in predicting healthy loans (label 0) with perfect precision, likely because the data is unbalanced so there are far more healthy loans than high-risk loans.  The model does a poor job in predicting `0` (high-risk loans) for comparison.  
Further analysis and consideration of alternative approaches may be necessary to improve the model's performance on high-risk loans. 

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [16]:
# Import the RandomOverSampler module from imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
ros = RandomOverSampler(random_state=1, sampling_strategy=0.5)

# Fit the original training data to the random_oversampler model
X_res, y_res = ros.fit_resample(X, y)

In [17]:
# Count the distinct values of the resampled labels data
np.unique(y_res)

array([0, 1])

In [18]:
print(Counter(y_res))

Counter({0: 75036, 1: 37518})


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [22]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_res = LogisticRegression(solver='lbfgs', random_state=1)
classifier_res

# Fit the model using the resampled training data
classifier_res.fit(X_res, y_res)

# Make a prediction using the testing data
predictions_res = classifier_res.predict(X_test)
pd.DataFrame({"Prediction": predictions_res, "Actual": y_test})

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
19379,0,0
19380,0,0
19381,0,0
19382,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [23]:
# Print the balanced_accuracy score of the model 
bal_acc_res=balanced_accuracy_score(y_test,predictions_res)
bal_acc_res

0.9929236578427081

In [24]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions_res)

array([[18651,   114],
       [    5,   614]])

In [25]:
# Print the classification report for the model
testing_report_res = classification_report(y_test, predictions_res)
print(testing_report_res)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model did well in terms of precision and f1-score on `0` (healthy-loans). 
The precision of the `1` (high-risk loan) labels went down by 0.01 (0.85 to 0.84). 
The f1-score went up by 0.03 (0.88 to 0.91). 
The recall score improved greatly for `1` (high-risk loan) (0.91 to 0.99).